## Week 2 Day 3

Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [78]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput, output_guardrail
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()

In [79]:
load_dotenv(override=True)

True

In [80]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key not set (and this is optional)
DeepSeek API Key exists and begins sk-
Groq API Key not set (and this is optional)


In [81]:
instructions1 = """You are a sales agent working for Domus Revita,
a company specialized in housing requialification projects that transform old houses big in two or three renewed housing units.
This mail is aimed to owner of big old houses in central or qualitative areas of the city.
The owner is probably strugling to sell the house as it is since it's not in a good condition an a lot of money is needed to requalify it.
You must explain that our team has very qualified architects and contractors that can help you to requalify your house.
The old big house will be partitioned in two housing units and completelly restructured.
One will be sold by our team and one will remain as payment to the owner that will have a new requalified house to sell, rent or live in.
IMPORTANT: generate the advertising in italian using max 300 words.
You write professional, serious cold emails."""

instructions2 = """You are a humorous, engaging sales agent working for Domus Revita, \
a company specialized in housing requialification projects that transform old houses big in two or three renewed housing units. \
This mail is aimed to owner of big old houses in central or qualitative areas of the city.
The owner is probably strugling to sell the house as it is since it's not in a good condition an a lot of money is needed to requalify it.
You must explain that our team has very qualified architects and contractors that can help you to requalify your house.
The old big house will be partitioned in two housing units and completelly restructured.
One will be sold by our team and one will remain as payment to the owner that will have a new requalified house to sell, rent or live in.
IMPORTANT: generate the advertising in italian using max 300 words.
You write witty, engaging cold emails that are likely to get a response."""

instructions3 = """You are a busy sales agent working for Domus Revita,
a company specialized in housing requialification projects that transform old houses big in two or three renewed housing units. \
This mail is aimed to owner of big old houses in central or qualitative areas of the city.
The owner is probably strugling to sell the house as it is since it's not in a good condition an a lot of money is needed to requalify it.
You must explain that our team has very qualified architects and contractors that can help you to requalify your house.
The old big house will be partitioned in two housing units and completelly restructured.
One will be sold by our team and one will remain as payment to the owner that will have a new requalified house to sell, rent or live in.
IMPORTANT: generate the advertising in italian using max 300 words.
You write concise, to the point cold emails."""

### It's easy to use any models with OpenAI compatible endpoints

IMPORTANT: Anthropic does not have a compatible endpoint as Gemini, DeepSeek and Groq (llama). 
To use Anthropic Claude is possible to use a third party provider called Open Router or better use the very powerfull Anthopic protocol MCP.

In [82]:
# These are compatible endpoints that allow to use OpenAI API with different providers
# GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
# GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [83]:

# AsyncOpenAI is a client that allows asynchronous API calls to OpenAI-compatible endpoints
# It takes a base_url parameter to specify the API endpoint and an api_key for authentication
# This enables using different AI providers that expose OpenAI-compatible APIs

# OpenAIChatCompletionsModel wraps the client to provide a standardized interface for chat completions
# It takes a model name and an OpenAI client instance
# This abstraction allows seamlessly switching between different providers while maintaining the same code structure

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
# gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
# groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
# gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
# llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [84]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 = Agent(name="OpenAI Sales Agent", instructions=instructions2, model="gpt-4o-mini") # Used OpenAI model to have an additional model to compare
# sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
# sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [85]:
description = "Write a cold sales email" # This is the description of the tool valid for all the similar tools

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
# tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [86]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("federico.tognetti@gmail.com")  # Change to your verified sender
    to_email = To("federico.tognetti@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [87]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response. \
IMPORTANT: Generate the subject in italian using max 20 words."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [88]:
email_tools = [subject_tool, html_tool, send_html_email]

In [89]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [90]:
tools = [tool1, tool2]
handoffs = [emailer_agent]

In [ ]:
# VERY IMPORTANT: pay attention to instrucrtions like the following to avoid LLM consuming too much time and money:
# "You can use the tools multiple times if you're not satisfied with the results from the first try."


sales_manager_instructions = """
You are a Sales Manager at Domus Revita. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself and keep the italian language.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to house owner from Domus Revita in italian language"

with trace("Automated commercial email by Domus Revita"):
    result = await Runner.run(sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

## Guardrails
Guardrails put a constraint to LLM to check consistency of to the user input and given output.
Guardrails are agents that perform a check and can be applied to the INPUT and / or to the OUTPUT; they cannot be put in the middle of the process.

In [91]:
# It defines the Pydantic model for the output of the guardrail agent
# In other words it defines the output of the guardrail agent
class NameCheckInput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckInput,
    model="gpt-4o-mini"
)

In [92]:
# This function serves as an input guardrail to check for personal names in messages
# It is decorated with @input_guardrail to register it as an input validation function
# Parameters:
#   - ctx: Context object containing execution context
#   - agent: The agent instance this guardrail is protecting
#   - message: The input message to check
# 
# The function:
# 1. Runs the guardrail_agent to analyze the message for personal names
# 2. Extracts whether a name was found and what the name was
# 3. Returns a GuardrailFunctionOutput with:
#    - output_info: Dictionary containing the found name
#    - tripwire_triggered: Boolean indicating if a name was found (true = block execution)
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output.name}, tripwire_triggered=is_name_in_message)

In [56]:
# TO BE NOTED:
# This is the same sales_manager agent but with:
# - same name
# - same instructions
# - same tools
# - same handoffs
# - same model
# In addition "input_guardrails" parameter is added to the agent to check the input message
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name] # This is the guardrail that will be used to check the input message
    )

message = "Send out a cold sales email addressed to house owner from Serena in italian language" # The guardrail will notice the name and block the execution

with trace("Protected automated commercial email by Domus Revita"):
    result = await Runner.run(careful_sales_manager, message)

# The excecution will produce and error message since guardrail will block the execution

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

## Check out the trace:

https://platform.openai.com/traces

In [ ]:

message = "Send out a cold sales email addressed to house owner from Head of Marketing in italian language"

with trace("Protected automated commercial email by Domus Revita"):
    result = await Runner.run(careful_sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>

## Excercise - Output guardrail

In [93]:
# This guardrail is an excercise to check if the final output includes the word "contractor"
# It will be use deepseek model to check the output
class ContractorCheckOutput(BaseModel):
    is_contractor_in_message: bool
    name: str

output_guardrail_agent = Agent( 
    name="Contractor check",
    instructions="Check if the output includes the word 'contractor', 'contractors', 'Contractor', 'Contractors'",
    output_type=ContractorCheckOutput,
    model=deepseek_model
)

@output_guardrail
async def guardrail_against_contractor(ctx, agent, output):
    result = await Runner.run(output_guardrail_agent, output)
    is_contractor_in_message = result.final_output.is_contractor_in_message
    return GuardrailFunctionOutput(output_info={"found_contractor": result.final_output.name}, tripwire_triggered=is_contractor_in_message)

super_careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name], # This is the guardrail that will be used to check the input and output messages
    output_guardrails=[guardrail_against_contractor] # This is the guardrail that will be used to check the output messages
    )

message = "Send out a cold sales email addressed to house owner from Head of Marketing in italian language" # The guardrail will notice the name and block the execution

with trace("Careful Input / output guardrail email by Domus Revita"):
    result = await Runner.run(super_careful_sales_manager, message)